## Proving Martry Wrong
Deck = {1=1, 2=2, ..., 10=Jack, 11=Queen, 12=King, 13=Ace}

### Marty's Strategy
>Pick the max/min card  
>Pick higher/lower depending on whether greater or less than 7
>Repeat, until done

### My Strategy

>Find # of cards at the extreme (see extrema cards below)  
>If, # of extrema cards == number_needed_right, chose the $(3-n_{RightRemaining})^{th}$ most extreme extrema  
>Else, choose the most extreme not extrema card   
>Pick higher/lower depending on whether greater or less than 7  
>Repeat  

* Example: Lets say you have a deck of {13, 12, 8, 7, 7, 7, ...}
    * If you currently need to get 3 right, pick the 8
    * If you currently need 2 right, pick the 12
    * If you need only 1 right, pick the best card, 13

In [1]:
EXTREMA_CARDS = {1,2, 12, 13}  # for my strategy

In [2]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.RandomState(42)  # seeding the random number generator

In [3]:
def marty_choose_card(table, n_right):
    # takes in table and outputs the location of the card we should go with
    relative_positions = abs(table - 7)  # sets all cards to be the distance away from 7 (so 13 or 1 look equally good)
    location = np.unravel_index(relative_positions.argmax(), table.shape)  # the location of the chosen card
    return location

In [4]:
def sean_choose_card(table, n_right):
    # takes in table and n_right, and outputs the location of the card we should go with
    current_extrema_cards = []
    current_extrema_cards_ind = []
    not_extrema_cards = []
    not_extrema_cards_ind = []
    # sorting all the cards into either extrema or non-extrema
    for i in range(TABLE_DIMENSIONS[0]):
        for j in range(TABLE_DIMENSIONS[1]):
            # if this location has an extrema card, record its value and location
            if table[i,j] in EXTREMA_CARDS:
                current_extrema_cards.append(table[i,j])
                current_extrema_cards_ind.append([i,j])
            else:
                not_extrema_cards.append(table[i,j])
                not_extrema_cards_ind.append([i,j])
    current_extrema_cards = np.array(current_extrema_cards)
    not_extrema_cards = np.array(not_extrema_cards)
    if current_extrema_cards.shape[0] >= 3 - n_right:
        # if we have have enough extrema cards, pick the (3-n_right)^th most extreme card
        relative_positions = abs(current_extrema_cards - 7)  # makes it so we are measuring distance from 7 (so max/min are equal)
        location = current_extrema_cards_ind[relative_positions.argsort()[n_right-3]]
    else:
        # we do not have enough extrema cards, so pick the most extreme "not-extrema" card
        relative_positions = abs(not_extrema_cards - 7)
        location = not_extrema_cards_ind[relative_positions.argmax()]
    return location

In [5]:
def make_guess(table, location, card_counts, rng):
    # takes in the location of the chosen card, the table, and the deck
    # makes a guess of higher/lower, draws a card from the deck with probabilities seen in card_counts
    # replaces the chosen card with the drawn card and returns the result of the guess, table, and card_counts
    i,j = location
    current_card = table[i, j]
    drawn_card = rng.choice(np.arange(1,14), p=card_counts/card_counts.sum())
    card_counts = decrement_card_counts(drawn_card, card_counts)
    if current_card >= 7:
        result = current_card > drawn_card
    else:
        result = current_card < drawn_card
    
    table[i, j] = drawn_card  # updating table
    return result, table, card_counts

In [6]:
def decrement_card_counts(card, card_counts):
    # updates the card probabilites
    card_index_in_counts = card-1
    card_counts[card_index_in_counts] -= 1
    if card_counts[card_index_in_counts] < 0:
        card_counts[card_index_in_counts] = 0
    if card_counts.sum() == 0:  # if we have run out of cards,
        print('Ran through the whole deck!')
        card_counts = np.full(shape=13, fill_value=4)  # reset the deck
    return card_counts

In [7]:
def play_game(card_pick_strategy, n_games, table_dim, rng=None):
    # runs the game, and returns a np array of the number of guesses required to beat each game
    if rng is None:  # if no random number generator is provided,
        rng = np.random.RandomState(42)
    n_guesses_over_games = []
    for game_number in range(n_games):
        # initializing game
        n_consecutive_right = 0
        n_guesses_in_game = 0
        game_remaining_card_counts = np.full(shape=13, fill_value=4)  # reset the card probabilities
        table = rng.randint(low=1, high=14, size=table_dim)  # draw a new table of cards
        for card in table.flatten():
            # updating the card probabilities from the cards in the table
            game_remaining_card_counts = decrement_card_counts(card, game_remaining_card_counts)
        n_consecutive_right = 0
        # starting game
        while n_consecutive_right < 3:
            n_guesses_in_game += 1
            picked_card = card_pick_strategy(table, n_consecutive_right)  # picks a card using the provided strategy
            result, table, game_remaining_card_counts = make_guess(table, picked_card, game_remaining_card_counts, rng)
            if result:  # if picked_card led to a correct guess
                n_consecutive_right += 1
            else:  # if picked_card is wrong, reset the n_consecutive_right
                n_consecutive_right = 0 
        # game end, recording results
        n_guesses_over_games.append(n_guesses_in_game)
    
    return np.array(n_guesses_over_games)

In [8]:
##HYPERPARAMETERS
TABLE_DIMENSIONS = [4,4]
N_GAMES = int(1e6)

In [9]:
# Playing game with Marty's strategy:
martys_results = play_game(marty_choose_card, n_games=N_GAMES, table_dim=TABLE_DIMENSIONS)

Ran through the whole deck!
Ran through the whole deck!
Ran through the whole deck!
Ran through the whole deck!


In [10]:
print('Marty\'s mean number of guesses per game:')
print(martys_results.mean())
print('Marty\'s standard deviation of guesses:')
print(martys_results.std())

Marty's mean number of guesses per game:
3.710797
Marty's standard deviation of guesses:
1.7158387525612657


In [11]:
# Playing game with Sean's strategy:
rng = np.random.RandomState(1)
seans_results = play_game(sean_choose_card, n_games=N_GAMES, table_dim=TABLE_DIMENSIONS)

In [12]:
print('Sean\'s mean number of guesses per game:')
print(seans_results.mean())
print('Sean\'s standard deviation of guesses:')
print(seans_results.std())

Sean's mean number of guesses per game:
3.555676
Sean's standard deviation of guesses:
1.25440112524822


In [13]:
if seans_results.mean() < martys_results.mean():
    print('Suck it Marty')
else:
    print('Must be a bug in the code')  # just for you marty lol

Suck it Marty
